In [ ]:
from geopandas import GeoDataFrame, GeoSeries
from shapely.geometry import Point, Polygon, MultiPolygon
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as ctx
import geopandas as gpd
import pandas as pd
import pyproj
import boto3

In [ ]:
# This will find the project directory so that data can be read in
directory = Path.cwd()
# Look up through parents until we find the base ladi-tutorial folder
matching_parent = [p for p in directory.parents if p.name.lower()=='ladi-tutorial']
if matching_parent:
    # if we find it, then it's the first entry
    tutorial_dir = matching_parent[0]
else:
    # otherwise, raise an error
    raise OSError(f'Notebook needs to be run from within the `ladi-tutorial` directory. Current directory is {str(directory)}')
print(tutorial_dir)

### The following datasets will be used in this analysis:

In [ ]:
# https://catalog.data.gov/dataset/tiger-line-shapefile-2017-nation-u-s-current-state-and-equivalent-national
file = tutorial_dir / 'data/Census-State/tl_2017_us_state.shp'
states = gpd.read_file(file)
# This line sets the CRS (Coordinate Reference System) so that all our maps will line up when plotting
states = states.to_crs(epsg=4326) 

# https://ais-faa.opendata.arcgis.com/datasets/e747ab91a11045e8b3f8a3efd093d3b5_0
file = tutorial_dir / 'data/FAA-Airports/Airports.shp'
airports = gpd.read_file(file)
airports = airports.to_crs(epsg=4326)

# https://catalog.data.gov/dataset/tiger-line-shapefile-2019-nation-u-s-current-metropolitan-statistical-area-micropolitan-statist
file = tutorial_dir / 'data/Census-CBSA/tl_2019_us_cbsa.shp'
us_cbsa = gpd.read_file(file)
us_cbsa = us_cbsa.to_crs(epsg=4326)

# http://www2.census.gov/programs-surveys/ahs/2017/AHS%202017%20National%20PUF%20v3.0%20Flat%20CSV.zip?#
file = tutorial_dir / 'data/Census-AHS/ahs2017n.csv'
ahs_data = pd.read_csv(file, usecols=['OMB13CBSA', 'DPFLDINS'])

### Functions:

In [ ]:
def meters_to_degrees(distance_meters):
    
    '''https://sciencing.com/convert-distances-degrees-meters-7858322.html (111,139)'''
    
    distance_degrees = (distance_meters / 111194.926644559) # number derived from matlab calculations
    return distance_degrees

In [ ]:
def df_to_gdf(df, what_on): 
    
    '''df_to_gdf merges a given DataFrame with image_metadata_gdf on whatever column name they choose'''
    
    merged_df = df.merge(image_metadata_gdf, on=what_on)
    clean_df = merged_df.dropna()
    gdf = GeoDataFrame(clean_df, crs='epsg:4326')
    
    return gdf

In [ ]:
def state_finder(df):  
    
    '''state_finder performs a spacial join with the states shapefile to find what state points are plotted in'''
    
    label_by_state = gpd.sjoin(df, states, op='within')
    label_by_state = label_by_state.drop(columns=['index_right'])
    
    return(label_by_state)

In [ ]:
def get_column_values(dataset_name, dataset_column): 
    
    '''get_column_values returns a dictionary that tells us how many of each value was found for a given column in a dataset'''
    
    column_dictionary_counter = {} 
    values = dataset_name[dataset_column].tolist()
    for i in values:
        if i not in column_dictionary_counter:
            column_dictionary_counter[i] = values.count(i)
            
    return(column_dictionary_counter)

In [ ]:
def months_with_images(state_abbreviation, label, year): 
    
    '''This function returns a dictionary telling the user how many photos were taken each month for the given dataset'''
    
    # Merge the images with the states to get a state name for each image
    images_by_state = state_finder(label)

    # Find images that were taken during the given timestamp
    state_images = images_by_state[images_by_state.STUSPS == state_abbreviation]
    state_images_timestamp = state_images[state_images['timestamp'].str.contains(year, na=False, case=True)]
    
    # Create a dictionary with the month names as keys and how many images were taken as their values
    months = {}
    for i in range(len(state_images_timestamp.timestamp)):
        month = pd.Timestamp(state_images_timestamp.timestamp.iloc[i])
        if month.month_name() not in months:
            state_images_per_month = 0
            
            for j in range(len(state_images_timestamp.timestamp)):
                month_counter = pd.Timestamp(state_images_timestamp.timestamp.iloc[j])
                if month.month_name() == month_counter.month_name():
                    state_images_per_month += 1
                months[month.month_name()] = state_images_per_month
    return(print(months))

In [ ]:
def months_with_images(state_abbreviation, label, year): 
    
    '''This function returns a dictionary telling the user how many photos were taken each month for the given dataset'''
    
    # Merge the images with the states to get a state name for each image
    images_by_state = state_finder(label)

    # Find images that were taken during the given timestamp
    state_images = images_by_state[images_by_state.STUSPS == state_abbreviation]
    state_images_timestamp = state_images[state_images['timestamp'].str.contains(year, na=False, case=True)]
    
    # Create a dictionary with the month names as keys and how many images were taken as their values
    months = {}
    for i in range(len(state_images_timestamp.timestamp)):
        month = pd.Timestamp(state_images_timestamp.timestamp.iloc[i])
        if month.month_name() not in months:
            state_images_per_month = 0
            
            for j in range(len(state_images_timestamp.timestamp)):
                month_counter = pd.Timestamp(state_images_timestamp.timestamp.iloc[j])
                if month.month_name() == month_counter.month_name():
                    state_images_per_month += 1
                months[month.month_name()] = state_images_per_month
    return(print(months))

In [ ]:
def images_per_cbsa(gdf, cbsa): 
    
    '''images_per_cbsa tells us how many points from your GeoDataFrame were found within each cbsa code'''
    
    image_counter = {}
    
    # Perform a spacial join between cbsa codes and the given GeoDataFrame
    images_within_cbsa = gpd.sjoin(gdf, cbsa, how='left', op='within')
    images = images_within_cbsa.NAMELSAD.tolist()

    # Create a dictionary with the CBSA code names as keys and how many images were taken as their values
    for i in images:
        if i not in image_counter:
            image_counter[i] = images.count(i)
    
    # make the dictionary into a dataframe 
    image_counter = pd.DataFrame.from_dict(image_counter, orient='index')
    
    
    return image_counter

### Here we read in the image metadata from the LADI s3 bucket. We will use this data along with our human and machine labels to make a geospatial analysis of the destruction from Hurricanes Michael and Florence.

In [ ]:
bucket_name = 'ladi'
file_1_path = 'Labels/ladi_images_metadata.csv'
client = boto3.client('s3')

obj_1 = client.get_object(Bucket = bucket_name, Key = file_1_path)

image_metadata = pd.read_csv(obj_1['Body'])
image_metadata_renamed = image_metadata.rename(columns={"uuid": "image_uuid"})
image_metadata_clean = image_metadata_renamed.dropna()

latitude = image_metadata_clean['gps_lat'].tolist() 
longitude = image_metadata_clean['gps_lon'].tolist()

#This line converts the DF to a GDF and sets the proper crs
image_metadata_gdf = GeoDataFrame(image_metadata_clean, crs='epsg:4326', geometry=gpd.points_from_xy(longitude, latitude))


In [ ]:
image_metadata_gdf

### Now we read in the LADI human labels dataset, specifically those with the label 'damage' or 'flood' because we want to see how many of the images taken actually contain damage

In [ ]:
#################### CLEAN AND VALIDATE LADI HUMAN LABELS #########################
human_label_filepath = "Labels/ladi_aggregated_responses_url.tsv"
obj_2 = client.get_object(Bucket = bucket_name, Key = human_label_filepath)
human_label_file = pd.read_csv(obj_2['Body'],sep = '\t' )

#STRIP OFF BRACKET AND COMMA FROM THE ANSWER CATEGORY
human_label_file["Answer"] = human_label_file["Answer"].str.strip('[|]')
human_label_file["Answer"] = human_label_file["Answer"].str.split(",",expand = True)

#EXTRACT LABELS WITH DAMAGE AND INFRASTRUCTURE CATEGORIES AND REMOVE THOSE LABELED 'NONE'
label_damage_infra = human_label_file[human_label_file['Answer'].str.contains('damage|infrastructure',na=False,case=False)]
label_clean = label_damage_infra[~label_damage_infra['Answer'].str.contains('none',na=False,case=False)]
human_flood_label = label_clean[label_clean['Answer'].str.contains('flood',na=False,case=False)]
human_damage_label = label_clean[label_clean['Answer'].str.contains('damage',na=False,case=False)]

### We can see that the human_damage_label dataset doesn't have alot of content so we will merge it with image_metadata_gdf to get all of the data we want for these specific images:

In [ ]:
human_damage_label.head()

In [ ]:
human_labeled_damage = df_to_gdf(human_damage_label, 'url')

In [ ]:
human_labeled_damage.head()

In [ ]:
human_flood_label.head()

In [ ]:
human_labeled_floods = df_to_gdf(human_flood_label, 'url')

In [ ]:
human_labeled_floods.head()

## Below is the code to read in the Ladi machine labels (~10 mins @ 100Mbps):

#################### CLEAN AND VALIDATE LADI MACHINE LABELS #########################

machine_label_filepath = "Labels/ladi_machine_labels.csv"
obj_3 = client.get_object(Bucket = bucket_name, Key = machine_label_filepath)
machine_flood_label = pd.read_csv(obj_3['Body'], usecols=['image_uuid', 'label_text'])
machine_flood_label_clean = machine_flood_label[machine_flood_label['label_text'].str.contains('flood', na=False,case=False)]

machine_labeled_floods = df_to_gdf(machine_flood_label_clean, 'image_uuid')

print(len(machine_labeled_floods))

lst = []
for i in machine_labeled_floods.timestamp:
    if timestamp.str.contains('2019'):
        lst.append(i)
print(len(lst))

### Now we will create a buffer (circle) around each of the airports at a given radius from the center changing the geometric Points to Polygons. We will use these buffer radius' to see what images are within a 5 mile range of one of the airports

In [ ]:
airports.geometry

In [ ]:
airports.geometry = airports.geometry.buffer(meters_to_degrees(8046.72)) #equal to 5 miles in meters
airports.geometry

## Now we can filter out images based on state, the year they were taken and the label dataset (if the condition is set to true it will plot airports and images that were taken within the buffer radius of the airport (default: 5 miles)):

In [ ]:
def state_plotter(state_abbreviation, df, year, conditional): 
    
    '''State plotter returns a map of the desired state and plots the points of flooding or disaster imagery 
       stored in the DataFrame and based on the given timeframe e.g. (year: '2018', month: '2018-10'). If 
       the conditional is True only images within the buffer radius of an airport will be plotted. Otherwise
       we will ignore all of the airports and plot all of the disaster images within the desired state'''
    
    if conditional is True:# Plot the Airports and only images within their buffer radius
        
        # ax is the matplotlib axis object. Setting this around the desired state will set the map boundary for the rest to follow 
        ax = states[states.STUSPS == state_abbreviation].plot(figsize=(10,10), alpha = .3, edgecolor = 'k')
        
        #Find the airports within the desired state
        airports_by_state = state_finder(airports) 
        state_airports = airports_by_state[airports_by_state.STUSPS == state_abbreviation]
        
        # Find the images given the state and the timestamp 
        images_by_state = state_finder(df)
        state_images = images_by_state[images_by_state.STUSPS == state_abbreviation]
        state_images_timestamp = state_images[state_images['timestamp'].str.contains(year, na=False, case=True)]
        
        # Plot the images that are within the buffer radius of an airport
        images_within_range = gpd.sjoin(state_images_timestamp, state_airports, op='within')
        images_within_range = images_within_range.drop(columns=['index_right'])
        images_within_range.plot(ax=ax, marker='.', markersize = 5, color='red', zorder=3)
        
        # Plot the Airports that have at least 1 image within range
        airports_within_range = gpd.sjoin(state_airports, state_images_timestamp, op='contains')
        airports_within_range.plot(ax=ax, color='black', alpha=.5, zorder=2)
        
        # Find the CBSA codes for the given State and plot them (STUSPS is the calumn name for state abbreviations)
        cbsa_by_state = state_finder(us_cbsa)
        state_cbsa = cbsa_by_state[cbsa_by_state['STUSPS'].str.contains(state_abbreviation, na=False, case=True)]
        state_cbsa.plot(ax=ax, alpha= .5, edgecolor = 'black', zorder=1)
      
        # Contextily has some nice basemaps this is how you would add one 
        ctx.add_basemap(ax, crs = state_cbsa.crs)
            
        # Find the CBSA code names and the number of images found within each as a DataFrame
        num_images_per_cbsa = images_per_cbsa(images_within_range, state_cbsa)
        count = len(images_within_range)
        
        return(plt.show(), print('Total images: ', count), print(num_images_per_cbsa)) # plt.show() overlays the maps
            
    else:
        
        # ax is the matplotlib axis object. Setting this around the state will set the map boundary for the rest to follow 
        ax = states[states.STUSPS == state_abbreviation].plot(figsize=(10,10), alpha = .3, edgecolor = 'k')
       
        # Find images in the given state taken during the given timeframe and plot these images
        images_by_state = state_finder(df)
        state_images = images_by_state[images_by_state.STUSPS == state_abbreviation]
        state_images_timestamp = state_images[state_images['timestamp'].str.contains(year, na=False, case=True)]
        state_images_timestamp.plot(ax=ax, marker='.', markersize = 5, color='red', zorder=3)
        
        # Find the CBSA codes for the given State and plot them (STUSPS is the calumn name for state abbreviations)
        cbsa_by_state = state_finder(us_cbsa)
        state_cbsa = cbsa_by_state[cbsa_by_state['STUSPS'].str.contains(state_abbreviation, na=False, case=True)]
        state_cbsa.plot(ax=ax, alpha= .5, edgecolor = 'black', zorder=1)
        
        # Contextily has some nice basemaps this is how you would add one 
        ctx.add_basemap(ax, crs = state_cbsa.crs)
        
        # Find the CBSA code names and the number of images found within each as a DataFrame
        num_images_per_cbsa = images_per_cbsa(state_images_timestamp, state_cbsa)
        count = len(state_images_timestamp)

        return(plt.show(), print('Total images: ', count), print(num_images_per_cbsa)) # plt.show() overlays the maps

# Hurricane Michael:
- MAKES LANDFALL IN FLORIDA ON OCTOBER 10TH 2018
- MICHAEL COMES UP THROUGH THE GULF COAST PUSHES NORTH THROUGH TALLAHASSE AND CONTINUES INTO GEORGIA
- WE CAN SEE THAT A MAJORITY OF THE IMAGES WERE TAKEN IN OCTOBER (THE SAME MONTH IT HIT) 
- ONLY ONE FOLLOW UP IMAGE WAS TAKEN IN NOVEMBER, 2018

In [ ]:
print('\n2018: ')
state_plotter('FL', image_metadata_gdf, '2018', True)

In [ ]:
months_with_images('FL', image_metadata_gdf, '2018') #returns the number of images taken in FL in 2018

In [ ]:
print('\n2018: ')
state_plotter('FL', image_metadata_gdf, '2018', False)
print('\nOCTOBER 2018: ')
state_plotter('FL', image_metadata_gdf, '2018-10', False)
#black circles represent the given buffer radius for the airports
#red dots are points for disaster images

In [ ]:
months_with_images('FL', human_labeled_floods, '2018')

In [ ]:
print('\n2018: ')
state_plotter('FL', human_labeled_floods, '2018', False)
print('\nOCTOBER 2018: ')
state_plotter('FL', human_labeled_floods, '2018-10', False) # returns only images taken in October 2018

In [ ]:
months_with_images('GA', image_metadata_gdf, '2018')

In [ ]:
print('\n2018: ')
state_plotter('GA', image_metadata_gdf, '2018', False)
print('\nOCTOBER 2018: ')
state_plotter('GA', image_metadata_gdf, '2018-10', False)

In [ ]:
months_with_images('GA', human_labeled_floods, '2018')

In [ ]:
print('2018: ')
state_plotter('GA', human_labeled_floods, '2018', False)
print('\nOCTOBER 2018: ')
state_plotter('GA', human_labeled_floods, '2018-10', False)

# Hurricane Florence:
- HIT CAROLINAS DIRECTLY ALONG COAST LINE (AUGUST 31, 2018-SEPTEMBER 18TH, 2018)
- MAKES LANDFALL IN NC ON SEPTEMBER 14TH
- PUSHES WESTWARD THROUGH THE CAROLINAS AND INTO GEORGIA


In [ ]:
months_with_images('NC', image_metadata_gdf, '2018')

In [ ]:
print('2018: ')
state_plotter('NC', image_metadata_gdf, '2018', False)
print('\nSEPTEMBER 2018: ')
state_plotter('NC', image_metadata_gdf, '2018-09', False)
print('\nOCTOBER 2018: ')
state_plotter('NC', image_metadata_gdf, '2018-10', False)
print('\nNOVEMBER 2018: ')
state_plotter('NC', image_metadata_gdf, '2018-11', False)
print('\nDECEMBER 2018: ')
state_plotter('NC', image_metadata_gdf, '2018-12', False)

In [ ]:
months_with_images('NC', human_labeled_floods, '2018')

In [ ]:
print('2018: ')
state_plotter('NC', human_labeled_floods, '2018', False)
print('\nSEPTEMBER 2018: ')
state_plotter('NC', human_labeled_floods, '2018-09', False)
print('\nOCTOBER 2018: ')
state_plotter('NC', human_labeled_floods, '2018-10', False)
print('\nNOVEMBER 2018: ')
state_plotter('NC', human_labeled_floods, '2018-11', False)
print('\nDECEMBER 2018: ')
state_plotter('NC', human_labeled_floods, '2018-12', False)

In [ ]:
months_with_images('SC', image_metadata_gdf, '2018')

In [ ]:
print('2018: ')
state_plotter('SC', image_metadata_gdf, '2018', False)

In [ ]:
months_with_images('SC', human_labeled_floods, '2018')

In [ ]:
print('2018: ')
state_plotter('SC', human_labeled_floods, '2018', False)
print('\nSEPTEMBER 2018: ')
state_plotter('SC', human_labeled_floods, '2018-09', False)
print('\nOCTOBER 2018: ')
state_plotter('SC', human_labeled_floods, '2018-10', False)

In [ ]:
months_with_images('FL', image_metadata_gdf, '2018')

In [ ]:
print('2018: ')
state_plotter('FL', image_metadata_gdf, '2018', False)
print('\nAUGUST 2018: ')
state_plotter('FL', image_metadata_gdf, '2018-08', False)
print('\nSEPTEMBER 2018: ')
state_plotter('FL', image_metadata_gdf, '2018-09', False)
print('\nOCTOBER 2018: ')
state_plotter('FL', image_metadata_gdf, '2018-10', False)
print('\nNOVEMBER 2018: ')
state_plotter('FL', image_metadata_gdf, '2018-11', False)
print('\nDECEMBER 2018: ')
state_plotter('FL', image_metadata_gdf, '2018-12', False)

In [ ]:
months_with_images('FL', human_labeled_floods, '2018')

In [ ]:
print('2018: ')
state_plotter('FL', human_labeled_floods, '2018', False)
print('\nOCTOBER 2018: ')
state_plotter('FL', human_labeled_floods, '2018-10', False)
print('\nNOVEMBER 2018: ')
state_plotter('FL', human_labeled_floods, '2018-11', False)